In [1]:
# nnUNetv2_predict \
#   -i /path/to/nnUNet_raw/DatasetXXX/imagesTs \
#   -o /path/to/nnunet_output \
#   -d DatasetXXX \
#   -c 3d_fullres \
#   -f all \
#   --save_probabilities



In [2]:
!python3 nnunet_to_medsam2_prompts.py \
    --nnunet_output nnunet_output \
    --ct_dir ct_images \
    --out_dir prompts_out \
    --dilation_iters 3 \
    --bbox_padding 5


Processing nnUNet outputs: 100%|██████████████████| 3/3 [00:47<00:00, 15.82s/it]

✔ Prompt generation complete.
  → JSON + coarse masks written to: prompts_out


In [3]:
# Minimal MedSAM2 inference: refine key-slice masks only
# Uses prompts from nnunet_to_medsam2_prompts.py and writes refined masks to medsam2_results
!python3 medsam2_infer_3D_CT_minimal.py \
    --checkpoint MedSAM2/checkpoints/MedSAM2_CTLesion.pt \
    --cfg configs/sam2.1_hiera_t512.yaml \
    --prompts_dir prompts_out \
    --output_dir medsam2_results

Using device: cpu for MedSAM2 inference
Found 3 prompt JSON files in prompts_out
Cases:   0%|                                              | 0/3 [00:00<?, ?it/s]
Processing case ct_1023
  Label LV: centroid=[343.9093422517222, 224.69845552517788, 79.12898493483044], coarse_mask=prompts_out/ct_1023_LV_mask.nii.gz
    Using key_slice_idx=79 based on coarse mask (z min=37, max=114, mean=79.1)
/Users/saschastocker/Documents/Stanford/Akshay/nnUNetandMedSAM/MedSAM2/sam2/sam2_video_predictor_npz.py:965: UserWarning: dlopen(/Users/saschastocker/Documents/Stanford/Akshay/nnUNetandMedSAM/MedSAM2/sam2/_C.so, 0x0002): tried: '/Users/saschastocker/Documents/Stanford/Akshay/nnUNetandMedSAM/MedSAM2/sam2/_C.so' (slice is not valid mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/saschastocker/Documents/Stanford/Akshay/nnUNetandMedSAM/MedSAM2/sam2/_C.so' (no such file), '/Users/saschastocker/Documents/Stanford/Akshay/nnUNetandMedSAM/MedSAM2/sam2/_C.so' (slice is not valid mach-o file)

Skippin

In [4]:
# Stitch individual MedSAM2 segmentations into combined multi-label files
!python3 stitch_medsam2_segmentations.py \
    --masks_dir medsam2_results \
    --output_dir medsam2_results \
    --reference_dir ct_images


Found 3 cases to process: ['ct_1023', 'ct_1028', 'ct_1030']
Processing cases:   0%|                                   | 0/3 [00:00<?, ?it/s]
Processing case: ct_1023
  Found 7 mask files:
    1: LV - ct_1023_LV_mask.nii.gz
    2: RV - ct_1023_RV_mask.nii.gz
    3: LA - ct_1023_LA_mask.nii.gz
    4: RA - ct_1023_RA_mask.nii.gz
    5: Myo - ct_1023_Myo_mask.nii.gz
    6: Aorta - ct_1023_Aorta_mask.nii.gz
    7: Pulmonary - ct_1023_Pulmonary_mask.nii.gz
  Using original CT as reference: ct_images/ct_1023.nii.gz
    Label 1 (LV): 5523 voxels
    Label 2 (RV): 11216 voxels
    Label 3 (LA): 15302 voxels
    Label 4 (RA): 21542 voxels
    Label 5 (Myo): 17941 voxels
    Label 6 (Aorta): 13775 voxels
    Label 7 (Pulmonary): 19614 voxels
  Saved with reference spatial info: medsam2_results/ct_1023_seg.nii.gz
  ✓ Combined segmentation saved: medsam2_results/ct_1023_seg.nii.gz
    Shape: (221, 512, 512)
    Labels present: [0, 1, 2, 3, 4, 5, 6, 7]
    Label mapping:
      0 = Background
      1